# Joining information

The idea here is to join info from different sources:
* RFEF, we have info from every match (referees, stadiuem, local and viisitor team, etc...
* LaLiga, for every match day and season we have the classification
We want to append the LaLiga information to every match collected in the RFEF, so that every team after each match day also has information regarding its position and points

In [166]:
import pandas as pd
import pprint

pp = pprint.PrettyPrinter(indent=2)

In [167]:
df_actas = pd.read_json('../rfef/actas.json') # rfef
df_clasif = pd.read_json('../laliga/laliga.json') #laliga

In [168]:
df_actas.head()

,arbitros,campeonato,campo,club_local,club_visitante,fecha,jornada,temporada,urls
0,{u'arbitro_asistente_1': [u'D. Fidalgo Trapote...,Campeonato de Liga 1ª División,El Madrigal,"{u'entrenador_2': u'Cousillas, Rubén Osvaldo',...","{u'entrenador_2': u'Carnero Gonzalez, Ramon', ...",2005-09-21 21:00:00,4,2005,[http://actas.rfef.es/actas/RFEF_CmpActa1?cod_...
1,"{u'arbitro_asistente_1': [u'D. Vidal Felani, E...",Campeonato de Liga 1ª División,La Rosaleda,"{u'entrenador_2': u'Granero Gil, Paulino', u't...","{u'entrenador_2': u'oliva Fornos, Juan Carlos'...",2005-09-21 21:00:00,4,2005,[http://actas.rfef.es/actas/RFEF_CmpActa1?cod_...
2,"{u'arbitro_asistente_1': [u'D. López Villate, ...",Campeonato de Liga 1ª División,Balaídos,"{u'entrenador_2': u'Carnero Gonzalez, Ramon', ...","{u'entrenador_2': u'Álvarez Giráldez, Antonio'...",2005-09-25 21:00:00,5,2005,[http://actas.rfef.es/actas/RFEF_CmpActa1?cod_...
3,"{u'arbitro_asistente_1': [u'D. Marañón Cantón,...",Campeonato de Liga 1ª División,Coliseum Alfonso Pérez,"{u'entrenador_2': u'Ruiz Pérez, Manuel', u'tar...","{u'entrenador_2': u'No Presenta', u'tarjetas':...",2005-09-21 21:00:00,4,2005,[http://actas.rfef.es/actas/RFEF_CmpActa1?cod_...
4,{u'arbitro_asistente_1': [u'D. Martínez Samani...,Campeonato de Liga 1ª División,Camp Nou,"{u'entrenador_2': u'Ten Cate, Hendrik Willem',...","{u'entrenador_2': u'Escriba Segura, Francisco ...",2005-09-21 21:00:00,4,2005,[http://actas.rfef.es/actas/RFEF_CmpActa1?cod_...


In [169]:
df_clasif.head()

,equipo,goles_contra,goles_favor,jornada,partidos_empatados,partidos_ganados,partidos_jugados,partidos_perdidos,posicion,puntos,temporada
0,R. Zaragoza,1,3,1,0,1,1,0,1,3,2004
1,Atlético de Madrid,0,2,1,0,1,1,0,2,3,2004
2,FC Barcelona,0,2,1,0,1,1,0,3,3,2004
3,Valencia CF,5,12,4,0,3,4,1,1,10,2004
4,FC Barcelona,2,9,4,0,3,4,1,2,10,2004


In [170]:
df_actas.columns

Index([u'arbitros', u'campeonato', u'campo', u'club_local', u'club_visitante',
       u'fecha', u'jornada', u'temporada', u'urls'],
      dtype='object')

In [171]:
jornada_list = sorted(df_actas['jornada'].unique().tolist())
temporada_list = sorted(df_actas['temporada'].unique().tolist())

### Automatically assigning points to each match based on the match result

In [172]:
equipos = []

for temporada in temporada_list:
    for jornada in jornada_list:
        df_partidos = df_actas[(df_actas.jornada==jornada) & (df_actas.temporada==temporada)]
        #partidos = [row.to_dict() for index, row in df_partidos.iterrows()]
        for index, row in df_partidos.iterrows():
            
            equipos.append(row.club_local['nombre'])
            try:
                # Empate
                if row.club_local["goles_count"] == row.club_visitante["goles_count"]:
                    #print "empate"
                    row.club_local["puntos_jornada"] = 1
                    row.club_visitante["puntos_jornada"] = 1

                # Victoria local
                elif row.club_local["goles_count"] > row.club_visitante["goles_count"]:
                    #print "victoria local"
                    row.club_local["puntos_jornada"] = 3
                    row.club_visitante["puntos_jornada"] = 0

                # Victoria visitante
                elif row.club_local["goles_count"] < row.club_visitante["goles_count"]:
                    #print "victoria visitante"
                    row.club_local["puntos_jornada"] = 0
                    row.club_visitante["puntos_jornada"] = 3    
                
            except:
                print index

In [173]:
#pp.pprint(df.iloc[0].club_local)
#pp.pprint(df.iloc[0].club_visitante)

In [174]:
# Distinct for a python list (does not preserve order)
equipos_list_actas = sorted(list(set(equipos)))
equipos_list_actas

[u'Albacete Balompi\xe9 SAD',
 u'Athletic Club',
 u'CD Legan\xe9s SAD',
 u'CD Numancia de Soria SAD',
 u'CD Tenerife SAD',
 u'Club Atl\xe9tico de Madrid SAD',
 u'Club Atl\xe9tico sasuna',
 u'C\xe1diz CF SAD',
 u'C\xf3rdoba CF SAD',
 u'D Almer\xeda SAD',
 u'D Las almas SAD',
 u'Deportivo Alav\xe9s SAD',
 u'FC Barcelona',
 u'Getafe CF SAD',
 u'Gimn\xe0stic de Tarragona SAD',
 u'Granada CF SAD',
 u'H\xe9rcules de Alicante CF SAD',
 u'Levante D SAD',
 u'M\xe1laga CF SAD',
 u'RC Celta de Vigo SAD',
 u'RC Deportivo de La Coru\xf1a SAD',
 u'RC Recreativo de Huelva SAD',
 u'RCD Mallorca SAD',
 u'RCD spanyol de Barcelona SAD',
 u'Rayo Vallecano de Madrid SAD',
 u'Real Betis Balompi\xe9 SAD',
 u'Real Madrid CF',
 u'Real Murcia CF SAD',
 u'Real Racing Club de Santander SAD',
 u'Real Sociedad de F\xfatbol SAD',
 u'Real Sporting de Gij\xf3n SAD',
 u'Real Valladolid CF SAD',
 u'Real Zaragoza SAD',
 u'SD ibar SAD',
 u'Sevilla FC SAD',
 u'Valencia CF SAD',
 u'Villarreal CF SAD',
 u'Xerez C.D. SAD',
 u

In [175]:
equipos_list_clasif = sorted(df_clasif.equipo.unique().tolist())
equipos_list_clasif

[u'Albacete B.',
 u'Athletic Club',
 u'Atl\xe9tico de Madrid',
 u'CA Osasuna',
 u'CD Legan\xe9s',
 u'CD Numancia',
 u'CD Tenerife',
 u'C\xe1diz CF',
 u'C\xf3rdoba CF',
 u'D. Alav\xe9s',
 u'Elche CF',
 u'FC Barcelona',
 u'Getafe CF',
 u'Granada CF',
 u'H\xe9rcules CF',
 u'Levante UD',
 u'M\xe1laga CF',
 u'N\xe0stic',
 u'R. Racing C.',
 u'R. Sporting',
 u'R. Valladolid CF',
 u'R. Zaragoza',
 u'RC Celta',
 u'RC Deportivo',
 u'RC Recreativo',
 u'RCD Espanyol',
 u'RCD Mallorca',
 u'Rayo Vallecano',
 u'Real Betis',
 u'Real Madrid',
 u'Real Murcia',
 u'Real Sociedad',
 u'SD Eibar',
 u'Sevilla FC',
 u'UD Almer\xeda',
 u'UD Las Palmas',
 u'Valencia CF',
 u'Villarreal CF',
 u'Xerez CD']

## Matching teams between different sources

Now that we have identified the teams from the df_actas, we need to find unique teams in df_clasif. Then we will establish the relationship between them.

In [176]:
#i = 0
#for equipo in equipos_list_actas:
#    print (i, equipo)
#    i += 1

#i = 0
#for equipo in equipos_list_clasif:
#    print (i, equipo)
#    i += 1

In [177]:
equipos_list = [
    (equipos_list_actas[0], equipos_list_clasif[0]),
    (equipos_list_actas[1], equipos_list_clasif[1]),
    (equipos_list_actas[2], equipos_list_clasif[4]),
    (equipos_list_actas[3], equipos_list_clasif[5]),
    (equipos_list_actas[4], equipos_list_clasif[6]),
    (equipos_list_actas[5], equipos_list_clasif[2]),
    (equipos_list_actas[6], equipos_list_clasif[3]),
    (equipos_list_actas[7], equipos_list_clasif[7]),
    (equipos_list_actas[8], equipos_list_clasif[8]),
    (equipos_list_actas[9], equipos_list_clasif[34]),
    (equipos_list_actas[10], equipos_list_clasif[35]),
    (equipos_list_actas[11], equipos_list_clasif[9]),
    (equipos_list_actas[12], equipos_list_clasif[11]),
    (equipos_list_actas[13], equipos_list_clasif[12]),
    (equipos_list_actas[14], equipos_list_clasif[17]),
    (equipos_list_actas[15], equipos_list_clasif[13]),
    (equipos_list_actas[16], equipos_list_clasif[14]),
    (equipos_list_actas[17], equipos_list_clasif[15]),
    (equipos_list_actas[18], equipos_list_clasif[16]),
    (equipos_list_actas[19], equipos_list_clasif[22]),
    (equipos_list_actas[20], equipos_list_clasif[23]),
    (equipos_list_actas[21], equipos_list_clasif[24]),
    (equipos_list_actas[22], equipos_list_clasif[26]),
    (equipos_list_actas[23], equipos_list_clasif[25]),
    (equipos_list_actas[24], equipos_list_clasif[27]),
    (equipos_list_actas[25], equipos_list_clasif[28]),
    (equipos_list_actas[26], equipos_list_clasif[29]),
    (equipos_list_actas[27], equipos_list_clasif[30]),
    (equipos_list_actas[28], equipos_list_clasif[18]),
    (equipos_list_actas[29], equipos_list_clasif[31]),
    (equipos_list_actas[30], equipos_list_clasif[19]),
    (equipos_list_actas[31], equipos_list_clasif[20]),
    (equipos_list_actas[32], equipos_list_clasif[21]),
    (equipos_list_actas[33], equipos_list_clasif[32]),
    (equipos_list_actas[34], equipos_list_clasif[33]),
    (equipos_list_actas[35], equipos_list_clasif[36]),
    (equipos_list_actas[36], equipos_list_clasif[37]),
    (equipos_list_actas[37], equipos_list_clasif[38]),
    (equipos_list_actas[38], equipos_list_clasif[10])
]

pp.pprint(equipos_list)

[ (u'Albacete Balompi\xe9 SAD', u'Albacete B.'),
  (u'Athletic Club', u'Athletic Club'),
  (u'CD Legan\xe9s SAD', u'CD Legan\xe9s'),
  (u'CD Numancia de Soria SAD', u'CD Numancia'),
  (u'CD Tenerife SAD', u'CD Tenerife'),
  (u'Club Atl\xe9tico de Madrid SAD', u'Atl\xe9tico de Madrid'),
  (u'Club Atl\xe9tico sasuna', u'CA Osasuna'),
  (u'C\xe1diz CF SAD', u'C\xe1diz CF'),
  (u'C\xf3rdoba CF SAD', u'C\xf3rdoba CF'),
  (u'D Almer\xeda SAD', u'UD Almer\xeda'),
  (u'D Las almas SAD', u'UD Las Palmas'),
  (u'Deportivo Alav\xe9s SAD', u'D. Alav\xe9s'),
  (u'FC Barcelona', u'FC Barcelona'),
  (u'Getafe CF SAD', u'Getafe CF'),
  (u'Gimn\xe0stic de Tarragona SAD', u'N\xe0stic'),
  (u'Granada CF SAD', u'Granada CF'),
  (u'H\xe9rcules de Alicante CF SAD', u'H\xe9rcules CF'),
  (u'Levante D SAD', u'Levante UD'),
  (u'M\xe1laga CF SAD', u'M\xe1laga CF'),
  (u'RC Celta de Vigo SAD', u'RC Celta'),
  (u'RC Deportivo de La Coru\xf1a SAD', u'RC Deportivo'),
  (u'RC Recreativo de Huelva SAD', u'RC Recreat

Linking the info from both dataframes

In [178]:
for temporada in temporada_list:
    for equipo_acta in equipos_list_actas:
        for jornada in jornada_list:
            obj_actas = df_actas[(df_actas.temporada==temporada) & (df_actas.jornada==jornada)]
            obj_clasif = df_clasif[(df_clasif.temporada==temporada) & (df_clasif.jornada==jornada)]
            for index, row in obj_actas.iterrows():
                
                if row.club_local['nombre'] == equipo_acta:
                    for x in equipos_list:
                        if x[0] == equipo_acta:
                            equipo_clasif = x[1]
                            for index_clasif, row_clasif in obj_clasif.iterrows():
                                if row_clasif.equipo == equipo_clasif:
                                    row.club_local["posicion"] = row_clasif.posicion
                                    row.club_local["puntos"] = row_clasif.puntos
                                    
                elif row.club_visitante['nombre'] == equipo_acta:
                    for x in equipos_list:
                        if x[0] == equipo_acta:
                            equipo_clasif = x[1]
                            for index_clasif, row_clasif in obj_clasif.iterrows():
                                if row_clasif.equipo == equipo_clasif:
                                    row.club_visitante["posicion"] = row_clasif.posicion
                                    row.club_visitante["puntos"] = row_clasif.puntos

Validation: checking Real Madrid's 2016 season

In [185]:
temporada = 2016
equipo = equipos_list_actas[26]

print "Team:",equipo, ", Season:",temporada

df_temp = df_actas[(df_actas.temporada==temporada)]
for jornada in jornada_list:
    obj_jornada = df_temp[df_temp.jornada==jornada]
    for index, row in obj_jornada.iterrows():

        if row.club_local['nombre'] == equipo:
            print "jornada:", jornada,', puntos:', row.club_local['puntos'],"( +", row.club_local['puntos_jornada'], '), posicion:', row.club_local['posicion']
            
        elif row.club_visitante['nombre'] == equipo:
            print "jornada:", jornada,', puntos:', row.club_visitante['puntos'],"( +", row.club_visitante['puntos_jornada'], '), posicion:', row.club_visitante['posicion']
    

Team: Real Madrid CF , Season: 2016
jornada: 1 , puntos: 3 ( + 3 ), posicion: 2
jornada: 2 , puntos: 6 ( + 3 ), posicion: 3
jornada: 3 , puntos: 9 ( + 3 ), posicion: 1
jornada: 4 , puntos: 12 ( + 3 ), posicion: 1
jornada: 5 , puntos: 13 ( + 1 ), posicion: 1
jornada: 6 , puntos: 14 ( + 1 ), posicion: 1
jornada: 7 , puntos: 15 ( + 1 ), posicion: 2
jornada: 8 , puntos: 18 ( + 3 ), posicion: 2
jornada: 9 , puntos: 21 ( + 3 ), posicion: 1
jornada: 10 , puntos: 24 ( + 3 ), posicion: 1
jornada: 11 , puntos: 27 ( + 3 ), posicion: 1
jornada: 12 , puntos: 30 ( + 3 ), posicion: 1
jornada: 13 , puntos: 33 ( + 3 ), posicion: 1
jornada: 14 , puntos: 34 ( + 1 ), posicion: 1
jornada: 15 , puntos: 37 ( + 3 ), posicion: 1
jornada: 16 , puntos: 37 ( + 0 ), posicion: 1
jornada: 17 , puntos: 40 ( + 3 ), posicion: 1
jornada: 18 , puntos: 40 ( + 3 ), posicion: 1
jornada: 19 , puntos: 43 ( + 3 ), posicion: 1
jornada: 20 , puntos: 46 ( + 3 ), posicion: 1
jornada: 21 , puntos: 46 ( + 3 ), posicion: 1
jornada: 2

### Output

In [191]:
df_actas.iloc[0].club_local

{u'alineacion': [[13, u'Viera, Mario Sebasti\xe1n'],
  [2, u'Rodr\xedguez Prado, Gonzalo Javier'],
  [3, u'Arruabarrena, Rodolfo Mart\xedn'],
  [5, u'Forl\xe1n Corazo, Diego'],
  [8, u'Riquelme, Juan Rom\xe1n'],
  [17, u'Rodr\xedguez Venta, Javier'],
  [18, u'Tacchinardi, Alessio'],
  [19, u'Senna Da Silva, Marcos Antonio'],
  [21, u'Cazorla Gonz\xe1lez, Santiago'],
  [22, u'Pe\xf1a Monta\xf1o, Juan Manuel'],
  [23, u'Romero Poy\xf3n, Jos\xe9 Mar\xeda']],
 u'entrenador': u'Pellegrini Ripamonti, Manuel Luis',
 u'entrenador_2': u'Cousillas, Rub\xe9n Osvaldo',
 u'goles': [{u'goleador': u'Riquelme, Juan Rom\xe1n',
   u'minuto': 81,
   u'tipo_gol': u'Gol'}],
 u'goles_count': 1,
 u'nombre': u'Villarreal CF SAD',
 'posicion': 19,
 'puntos': 2,
 'puntos_jornada': 0,
 u'suplentes': [[25, u'Barbosa, Mariano Sebastian'],
  [6, u'Moreno Verd\xfa, Jos\xe9 Joaqu\xedn'],
  [7, u'Betancort Perdomo, Antonio Guayre'],
  [9, u'Figueroa, Luciano Gabriel'],
  [10, u'Garc\xeda Junyent, Roger'],
  [16, u'\xc

In [205]:
df_output = df_actas[["temporada", "jornada", "fecha", "club_local", "club_visitante"]]
    
#df_output.drop('urls', axis=1, inplace=True)
#df_output.drop('campo', axis=1, inplace=True)
#df_output.drop('campeonato', axis=1, inplace=True)

for index, row in df_output.iterrows():
    
    row.club_local.pop("alineacion", None)
    row.club_local.pop("alineacion", None)
    row.club_local.pop("suplentes", None)
    row.club_local.pop("sustituciones", None)
    row.club_local.pop("entrenador", None)
    row.club_local.pop("entrenador_2", None)
    row.club_local.pop("arbitros", None)
    
    row.club_visitante.pop("alineacion", None)
    row.club_visitante.pop("alineacion", None)
    row.club_visitante.pop("suplentes", None)
    row.club_visitante.pop("sustituciones", None)
    row.club_visitante.pop("entrenador", None)
    row.club_visitante.pop("entrenador_2", None)
    row.club_visitante.pop("arbitros", None)

In [209]:
df_output.to_json('./football.json', orient='records')

## Deprecated

In [181]:
## Calculate puntos for each jornada for each team
#
#puntos = 0
#goles = 0
#i = 0
#for temporada in temporada_list:
#    for equipo in equipos_list:
#        puntos = 0
#        goles = 0
#        for jornada in jornada_list:
#            obj = df[(df.temporada==temporada) & (df.jornada==jornada)]
#            for index, row in obj.iterrows():
#                if row.club_local['nombre'] == equipo:
#                    puntos += row.club_local["puntos_jornada"]
#                    goles += row.club_local["goles_count"]
#                    row.club_local["puntos"] = puntos
#                    row.club_local["goles_count_acum"] = goles
#                elif row.club_visitante['nombre'] == equipo:
#                    puntos += row.club_visitante["puntos_jornada"]
#                    goles += row.club_visitante["goles_count"]
#                    row.club_visitante["puntos"] = puntos
#                    row.club_visitante["goles_count_acum"] = goles

In [182]:
## Comprobación
#
#i = 0
#a = df[(df.temporada == "Temporada 2003/2004")]
#equipo = "FC Barcelona"
#for jornada in jornada_list:
#    i += 1
#    if i < 15:
#        b = a[(a.jornada==jornada)]
#        for index, row in b.iterrows():
#            if row.club_local["nombre"]==equipo:
#                print jornada, "(+",row.club_local['puntos_jornada'],")",row.club_local['puntos'], "goles:", row.club_local['goles_count_acum']
#            elif row.club_visitante["nombre"]==equipo:
#                print jornada, "(+",row.club_visitante['puntos_jornada'],")", row.club_visitante['puntos'], "goles:", row.club_visitante['goles_count_acum']